In [0]:
import torch
import cv2
import sys
import numpy as np
import argparse
import torch.nn as nn
from google.colab import drive
from torch.autograd import Variable
from torch.autograd import Function
from torchvision import models
from torchvision import utils

In [2]:
# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


## Import depedences from google drive

In [3]:
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
%cd '/content/gdrive/My Drive/UnB/TCC-1/TCC1-1-dataset-final'

/content/gdrive/My Drive/UnB/TCC-1/TCC1-1-dataset-final


In [5]:
!ls

cm_data_set_exp1.png  grad_cam.py			skin.txt
dataset_final	      __pycache__			train_val_phase.csv
dataset_final.tar.xz  restnet_model152_trained_exp7.pt


## Load the skin lession model

In [0]:
PRE_MODEL_DIR='/content/gdrive/My Drive/UnB/TCC-1/TCC1-1-dataset-final/restnet_model152_trained_exp7.pt'

In [0]:
model_name='resnet'
num_classes = 9
feature_extract = False

In [0]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [9]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet152
        """
        model_ft = models.resnet152(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    else:
        print("Invalid model name, exiting...")
        exit()
    
    return model_ft, input_size

# Initialize the model for this run
model, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet152-b121ed2d.pth" to /root/.cache/torch/checkpoints/resnet152-b121ed2d.pth
100%|██████████| 230M/230M [00:25<00:00, 9.43MB/s]


In [10]:
if train_on_gpu:
    state = torch.load(PRE_MODEL_DIR)
else:
    state = torch.load(PRE_MODEL_DIR, map_location='cpu')

# Loading weights in restnet architecture
model.load_state_dict(state['state_dict'])

<All keys matched successfully>

In [11]:
classes_skin = state['class_to_idx']
classes_skin

{'actinic-keratosis': 0,
 'basal-cell-carcinoma': 1,
 'dermatofibroma': 2,
 'hemangioma': 3,
 'intraepithelial-carcinoma': 4,
 'malignant-melanoma': 5,
 'melanocytic-nevus': 6,
 'pyogenic-granuloma': 7,
 'squamous-cell-carcinoma': 8}

## Main function

In [0]:
from __future__ import print_function
import copy
import os.path as osp
import click
import cv2
import matplotlib.cm as cm
import numpy as np
import torch
import torch.hub
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision import models, transforms

from grad_cam import (
    BackPropagation,
    Deconvnet,
    GradCAM,
    GuidedBackPropagation,
    occlusion_sensitivity,
)

def get_device(cuda):
    cuda = cuda and torch.cuda.is_available()
    device = torch.device("cuda" if cuda else "cpu")
    if cuda:
        current_device = torch.cuda.current_device()
        print("Device:", torch.cuda.get_device_name(current_device))
    else:
        print("Device: CPU")
    return device


def get_classtable():
    classes = []
    with open("/content/gdrive/My Drive/UnB/TCC-1/TCC1-1-dataset-final/skin.txt") as lines:
        for line in lines:
            line = line.strip().split(" ", 1)[1]
            line = line.split(", ", 1)[0].replace(" ", "_")
            classes.append(line)
    return classes


def preprocess(image_path):
    raw_image = cv2.imread(image_path)
    raw_image = cv2.resize(raw_image, (224, 224))
    image = transforms.Compose(
        [
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            transforms.Resize((256, 256)),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
        ]
    )(raw_image[..., ::-1].copy())
    return image, raw_image


def save_gradient(filename, gradient):
    gradient = gradient.cpu().numpy().transpose(1, 2, 0)
    gradient -= gradient.min()
    gradient /= gradient.max()
    gradient *= 255.0
    cv2.imwrite(filename, np.uint8(gradient))


def save_gradcam(filename, gcam, raw_image, paper_cmap=False):
    gcam = gcam.cpu().numpy()
    cmap = cm.jet_r(gcam)[..., :3] * 255.0
    if paper_cmap:
        alpha = gcam[..., None]
        gcam = alpha * cmap + (1 - alpha) * raw_image
    else:
        gcam = (cmap.astype(np.float) + raw_image.astype(np.float)) / 2
    cv2.imwrite(filename, np.uint8(gcam))


def save_sensitivity(filename, maps):
    maps = maps.cpu().numpy()
    scale = max(maps[maps > 0].max(), -maps[maps <= 0].min())
    maps = maps / scale * 0.5
    maps += 0.5
    maps = cm.bwr_r(maps)[..., :3]
    maps = np.uint8(maps * 255.0)
    maps = cv2.resize(maps, (224, 224), interpolation=cv2.INTER_NEAREST)
    cv2.imwrite(filename, maps)

In [0]:
def demo3(image_paths, topk, output_dir, cuda):
    """
    Generate Grad-CAM with original models
    """

    device = get_device(cuda)

    # Synset words
    classes = get_classtable()

    # Third-party model from my other repository, e.g. Xception v1 ported from Keras
    model.to(device)
    model.eval()

    # Check available layer names
    print("Layers:")
    for m in model.named_modules():
        print("\t", m[0])

    # Here we choose the last convolution layer
    target_layer = "exit_flow.conv4"

    # Preprocessing
    def _preprocess(image_path):
        raw_image = cv2.imread(image_path)
        raw_image = cv2.resize(raw_image, model.image_shape)
        image = torch.FloatTensor(raw_image[..., ::-1].copy())
        image -= model.mean
        image /= model.std
        image = image.permute(2, 0, 1)
        return image, raw_image

    # Images
    images = []
    raw_images = []
    print("Images:")
    for i, image_path in enumerate(image_paths):
        print("\t#{}: {}".format(i, image_path))
        image, raw_image = _preprocess(image_path)
        images.append(image)
        raw_images.append(raw_image)
    images = torch.stack(images).to(device)

    print("Grad-CAM:")

    gcam = GradCAM(model=model)
    probs, ids = gcam.forward(images)

    for i in range(topk):

        # Grad-CAM
        gcam.backward(ids=ids[:, [i]])
        regions = gcam.generate(target_layer=target_layer)

        for j in range(len(images)):
            print("\t#{}: {} ({:.5f})".format(j, classes[ids[j, i]], probs[j, i]))

            # Grad-CAM
            save_gradcam(
                filename=osp.join(
                    output_dir,
                    "{}-{}-gradcam-{}-{}.png".format(
                        j, "xception_v1", target_layer, classes[ids[j, i]]
                    ),
                ),
                gcam=regions[j, 0],
                raw_image=raw_images[j],
            )

In [0]:
def demo1(image_paths, target_layer, arch, topk, output_dir, cuda):
    """
    Visualize model responses given multiple images
    """

    device = get_device(cuda)

    # Synset words
    classes = get_classtable()

    # Model from torchvision
    model = models.__dict__[arch](pretrained=True)
    model.to(device)
    model.eval()

    # Images
    images = []
    raw_images = []
    print("Images:")
    for i, image_path in enumerate(image_paths):
        print("\t#{}: {}".format(i, image_path))
        image, raw_image = preprocess(image_path)
        images.append(image)
        raw_images.append(raw_image)
    images = torch.stack(images).to(device)

    """
    Common usage:
    1. Wrap your model with visualization classes defined in grad_cam.py
    2. Run forward() with images
    3. Run backward() with a list of specific classes
    4. Run generate() to export results
    """

    # =========================================================================
    print("Vanilla Backpropagation:")

    bp = BackPropagation(model=model)
    probs, ids = bp.forward(images)

    for i in range(topk):
        # In this example, we specify the high confidence classes
        bp.backward(ids=ids[:, [i]])
        gradients = bp.generate()

        # Save results as image files
        for j in range(len(images)):
            print("\t#{}: {} ({:.5f})".format(j, classes[ids[j, i]], probs[j, i]))

            save_gradient(
                filename=osp.join(
                    output_dir,
                    "{}-{}-vanilla-{}.png".format(j, arch, classes[ids[j, i]]),
                ),
                gradient=gradients[j],
            )

    # Remove all the hook function in the "model"
    bp.remove_hook()

    # =========================================================================
    print("Deconvolution:")

    deconv = Deconvnet(model=model)
    _ = deconv.forward(images)

    for i in range(topk):
        deconv.backward(ids=ids[:, [i]])
        gradients = deconv.generate()

        for j in range(len(images)):
            print("\t#{}: {} ({:.5f})".format(j, classes[ids[j, i]], probs[j, i]))

            save_gradient(
                filename=osp.join(
                    output_dir,
                    "{}-{}-deconvnet-{}.png".format(j, arch, classes[ids[j, i]]),
                ),
                gradient=gradients[j],
            )

    deconv.remove_hook()

    # =========================================================================
    print("Grad-CAM/Guided Backpropagation/Guided Grad-CAM:")

    gcam = GradCAM(model=model)
    _ = gcam.forward(images)

    gbp = GuidedBackPropagation(model=model)
    _ = gbp.forward(images)

    for i in range(topk):
        # Guided Backpropagation
        gbp.backward(ids=ids[:, [i]])
        gradients = gbp.generate()

        # Grad-CAM
        gcam.backward(ids=ids[:, [i]])
        regions = gcam.generate(target_layer=target_layer)

        for j in range(len(images)):
            print("\t#{}: {} ({:.5f})".format(j, classes[ids[j, i]], probs[j, i]))

            # Guided Backpropagation
            save_gradient(
                filename=osp.join(
                    output_dir,
                    "{}-{}-guided-{}.png".format(j, arch, classes[ids[j, i]]),
                ),
                gradient=gradients[j],
            )

            # Grad-CAM
            save_gradcam(
                filename=osp.join(
                    output_dir,
                    "{}-{}-gradcam-{}-{}.png".format(
                        j, arch, target_layer, classes[ids[j, i]]
                    ),
                ),
                gcam=regions[j, 0],
                raw_image=raw_images[j],
            )

            # Guided Grad-CAM
            save_gradient(
                filename=osp.join(
                    output_dir,
                    "{}-{}-guided_gradcam-{}-{}.png".format(
                        j, arch, target_layer, classes[ids[j, i]]
                    ),
                ),
                gradient=torch.mul(regions, gradients)[j],
            )

In [21]:
img = '/content/gdrive/My Drive/UnB/TCC-1/TCC1-1-dataset-final/dataset_final/test/actinic-keratosis/1604.jpg'

demo1(image_paths=img, target_layer='layer4.2', arch='resnet152', topk=5, output_dir='./results', cuda='cuda')

Device: Tesla P100-PCIE-16GB
Images:
	#0: /


error: ignored

In [23]:
demo3(image_paths=img, topk=3, output_dir='./results', cuda='cuda')

Device: Tesla P100-PCIE-16GB
Layers:
	 
	 conv1
	 bn1
	 relu
	 maxpool
	 layer1
	 layer1.0
	 layer1.0.conv1
	 layer1.0.bn1
	 layer1.0.conv2
	 layer1.0.bn2
	 layer1.0.conv3
	 layer1.0.bn3
	 layer1.0.relu
	 layer1.0.downsample
	 layer1.0.downsample.0
	 layer1.0.downsample.1
	 layer1.1
	 layer1.1.conv1
	 layer1.1.bn1
	 layer1.1.conv2
	 layer1.1.bn2
	 layer1.1.conv3
	 layer1.1.bn3
	 layer1.1.relu
	 layer1.2
	 layer1.2.conv1
	 layer1.2.bn1
	 layer1.2.conv2
	 layer1.2.bn2
	 layer1.2.conv3
	 layer1.2.bn3
	 layer1.2.relu
	 layer2
	 layer2.0
	 layer2.0.conv1
	 layer2.0.bn1
	 layer2.0.conv2
	 layer2.0.bn2
	 layer2.0.conv3
	 layer2.0.bn3
	 layer2.0.relu
	 layer2.0.downsample
	 layer2.0.downsample.0
	 layer2.0.downsample.1
	 layer2.1
	 layer2.1.conv1
	 layer2.1.bn1
	 layer2.1.conv2
	 layer2.1.bn2
	 layer2.1.conv3
	 layer2.1.bn3
	 layer2.1.relu
	 layer2.2
	 layer2.2.conv1
	 layer2.2.bn1
	 layer2.2.conv2
	 layer2.2.bn2
	 layer2.2.conv3
	 layer2.2.bn3
	 layer2.2.relu
	 layer2.3
	 layer2.3.conv1
	 lay

AttributeError: ignored

In [44]:
!python main.py demo1 -a resnet152 \
                     -t layer4.2 \
                     -k 5 \
                     -o "/content/gdrive/My Drive/UnB/TCC-1/TCC1-1-dataset-final/grand-cam-results" \
                     -i "/content/gdrive/My Drive/UnB/TCC-1/TCC1-1-dataset-final/dataset_final/test/actinic-keratosis/1604.jpg"


KeyboardInterrupt: ignored

In [49]:
!python main.py demo3 -o "/content/gdrive/My Drive/UnB/TCC-1/TCC1-1-dataset-final/grand-cam-results" \
                      -i "/content/gdrive/My Drive/UnB/TCC-1/TCC1-1-dataset-final/dataset_final/test/actinic-keratosis/1604.jpg"


Mode: demo3
Device: Tesla P100-PCIE-16GB
CUDA is available!  Training on GPU ...
Device: Tesla P100-PCIE-16GB
Layers:
	 
	 conv1
	 bn1
	 relu
	 maxpool
	 layer1
	 layer1.0
	 layer1.0.conv1
	 layer1.0.bn1
	 layer1.0.conv2
	 layer1.0.bn2
	 layer1.0.conv3
	 layer1.0.bn3
	 layer1.0.relu
	 layer1.0.downsample
	 layer1.0.downsample.0
	 layer1.0.downsample.1
	 layer1.1
	 layer1.1.conv1
	 layer1.1.bn1
	 layer1.1.conv2
	 layer1.1.bn2
	 layer1.1.conv3
	 layer1.1.bn3
	 layer1.1.relu
	 layer1.2
	 layer1.2.conv1
	 layer1.2.bn1
	 layer1.2.conv2
	 layer1.2.bn2
	 layer1.2.conv3
	 layer1.2.bn3
	 layer1.2.relu
	 layer2
	 layer2.0
	 layer2.0.conv1
	 layer2.0.bn1
	 layer2.0.conv2
	 layer2.0.bn2
	 layer2.0.conv3
	 layer2.0.bn3
	 layer2.0.relu
	 layer2.0.downsample
	 layer2.0.downsample.0
	 layer2.0.downsample.1
	 layer2.1
	 layer2.1.conv1
	 layer2.1.bn1
	 layer2.1.conv2
	 layer2.1.bn2
	 layer2.1.conv3
	 layer2.1.bn3
	 layer2.1.relu
	 layer2.2
	 layer2.2.conv1
	 layer2.2.bn1
	 layer2.2.conv2
	 layer2.2.bn2


In [51]:
!python main.py demo4 -a resnet152 \
                      -o "/content/gdrive/My Drive/UnB/TCC-1/TCC1-1-dataset-final/grand-cam-results" \
                      -i "/content/gdrive/My Drive/UnB/TCC-1/TCC1-1-dataset-final/dataset_final/test/actinic-keratosis/1604.jpg"


Mode: demo4
Device: Tesla P100-PCIE-16GB
CUDA is available!  Training on GPU ...
Device: Tesla P100-PCIE-16GB
Images:
	#0: /content/gdrive/My Drive/UnB/TCC-1/TCC1-1-dataset-final/dataset_final/test/actinic-keratosis/1604.jpg
Occlusion Sensitivity:
Patch: 10
	#0: basal-cell-carcinoma (0.77285)
Patch: 15
	#0: basal-cell-carcinoma (0.77285)
Patch: 25
	#0: basal-cell-carcinoma (0.77285)
Patch: 35
 30% 116/392 [01:20<03:12,  1.43it/s]
Aborted!
Exception ignored in: <bound method tqdm.__del__ of  30% 116/392 [01:20<03:12,  1.43it/s]>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/tqdm/_tqdm.py", line 931, in __del__
    self.close()
  File "/usr/local/lib/python3.6/dist-packages/tqdm/_tqdm.py", line 1133, in close
    self._decr_instances(self)
  File "/usr/local/lib/python3.6/dist-packages/tqdm/_tqdm.py", line 496, in _decr_instances
    cls.monitor.exit()
  File "/usr/local/lib/python3.6/dist-packages/tqdm/_monitor.py", line 52, in exit
    self.join()
  

In [57]:
!python main.py demo2 -o "/content/gdrive/My Drive/UnB/TCC-1/TCC1-1-dataset-final/grand-cam-results" \
                      -i "/content/gdrive/My Drive/UnB/TCC-1/TCC1-1-dataset-final/dataset_final/test/actinic-keratosis/teste.jpg"

Mode: demo2
Device: Tesla P100-PCIE-16GB
CUDA is available!  Training on GPU ...
Device: Tesla P100-PCIE-16GB
Images:
	#0: /content/gdrive/My Drive/UnB/TCC-1/TCC1-1-dataset-final/dataset_final/test/actinic-keratosis/teste.jpg
Generating Grad-CAM @relu
	#0: basal-cell-carcinoma (0.77285)
Generating Grad-CAM @layer1
	#0: basal-cell-carcinoma (0.77285)
Generating Grad-CAM @layer2
	#0: basal-cell-carcinoma (0.77285)
Generating Grad-CAM @layer3
	#0: basal-cell-carcinoma (0.77285)
Generating Grad-CAM @layer4
	#0: basal-cell-carcinoma (0.77285)
